In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import numpy as np
from pyspark.mllib.linalg.distributed import *
from pyspark.mllib.linalg import *
from scipy.sparse import *
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [3]:
icm = sc.textFile("../icm.csv")
icm = icm.map(lambda l: l.split(','))\
            .filter(lambda line: line[0] != 'itemId')\
            .map(lambda x: (int(x[0]), int(x[1]), 1))
trainSet = sc.textFile("../train.csv")
trainSet = trainSet.map(lambda l: l.split(','))
trainSet = trainSet.filter(lambda line: 'userId' not in line)
trainSet = trainSet.map(lambda line: (int(line[0]), int(line[1]), float(line[2])))

#pipelinedTrainsetfromUB
trainSetUB = sc.textFile("../trainPipelined2.csv")
trainSetUB = trainSetUB.map(lambda l: l.split(','))
trainSetUB = trainSetUB.filter(lambda line: 'userId' not in line)
trainSetUB = trainSetUB.map(lambda line: (int(line[0]), int(line[1]), float(line[2])))

seenItems= trainSet.map(lambda x: (x[0],[x[1]])).reduceByKey(lambda x,y: x + y)
seenItemsDict=seenItems.collectAsMap()

trainSet = trainSet.union(trainSetUB)
#FOR THE TOP POP
itemSet = trainSet.map(lambda x: (x[1], x[2]))
itemsCount = trainSet.map(lambda x: (x[1],1)).reduceByKey(lambda x,y : x + y)
itemsCount_dict = itemsCount.collectAsMap()
#-----------------------------------------
featureFreq = icm.map(lambda x: (x[1],1)).reduceByKey(lambda x, y: x + y)
featureFreqDict = featureFreq.collectAsMap()
prodCount= icm.map(lambda x: x[0]).distinct().count()
featureIdf = featureFreq.map(lambda x: (x[0],np.log10(prodCount/x[1])))
featureIdfDict = featureIdf.collectAsMap()

targetUsers = sc.textFile("../target_user.csv").filter(lambda x: "userId" not in x).map(lambda x: int(x))
targets=targetUsers.collect()

######
#TEST COMPUTING THE PREDICTION NORMALIZING BY THE N OF FEATURES AND NOT THE SQRT OF IT
norms = icm.map(lambda x: (x[0],1))\
                .reduceByKey(lambda x, y: x+y).mapValues(lambda x: np.sqrt(x))\
                .collectAsMap()

normalized = icm.map(lambda x: (x[0], x[1], x[2]/norms[x[0]]))

In [4]:
print(trainSetUB.count())

172674


In [5]:
#Just consider the row of the users to predict
#IF you0re asking, only the idf depends on all the training set, in fact it is computed before reducing the trainset
trainSet=trainSet.filter(lambda x: x[0] in targets)
print(trainSet.count())
data = trainSet.map(lambda x: x[2]).collect()
rows = trainSet.map(lambda x: x[0]).collect()
cols = trainSet.map(lambda x: x[1]).collect()
data.append(0)
rows.append(15364)
cols.append(37142)
userItem=csr_matrix((data,(rows,cols)))
print("userItem shape:",userItem.shape)
data = normalized.map(lambda x: x[2]).collect()
rows = normalized.map(lambda x: x[0]).collect()
cols = normalized.map(lambda x: x[1]).collect()
data.append(0)
rows.append(37142)
cols.append(80)
itemFeature = csc_matrix((data,(rows,cols)))
print("itemFeat shape:",itemFeature.shape)

90981
userItem shape: (15365, 37143)
itemFeat shape: (37143, 19716)


In [6]:
userFeature = userItem.dot(itemFeature)
userFeature.shape

(15365, 19716)

In [7]:
data = []
rows = []
cols = []
for f in featureIdfDict.keys():
    data.append(featureIdfDict[f])
    cols.append(f)
    rows.append(f)
featureIdf = csr_matrix((data,(rows,cols)))
featureIdf.shape

(19716, 19716)

In [8]:
userProfile = userFeature.dot(featureIdf)
prediction = userProfile.dot(itemFeature.transpose())
prediction.shape

(15365, 37143)

In [9]:
numberOfRecommendations=5
#TOP POPULAR
cost=6
avgRatings=itemSet.reduceByKey(lambda x,y: x+y)
avgRatings=avgRatings.map(lambda x: (x[0],x[1]/(itemsCount_dict[x[0]]+cost)))
avgRatings.take(5)
itemOrderByPop=avgRatings.sortBy(lambda x: x[1], ascending=False)
itemPop = np.array(itemOrderByPop.map(lambda x: x[0]).collect())
#--------------------------------------------------------------

def recommendTopPop(user_id, removeSeen=True):
    seenItems = np.array(seenItemsDict[user_id])
    recommendedList = itemPop
    if(removeSeen):
        unseen_mask = np.in1d(recommendedList, seenItems, invert=True)
        recommendedList = recommendedList[unseen_mask]       
    return recommendedList[:numberOfRecommendations]


def fillWithTopPop(recommended,user):
    TopPop=recommendTopPop(user)
    for i in range (numberOfRecommendations-len(recommended)):
        recommended.append(TopPop[i])
    return recommended


def getRecommended(user):
    recommended = []
    itemsPred = prediction.getrow(user).toarray()[0]

    for i in range(0,len(itemsPred)):
        if(itemsPred[i]!=0):
                if i not in seenItemsDict[user]:
                    recommended.append((i, itemsPred[i]))
    recommended.sort(key = lambda x: -x[1])
    recommended=recommended[:numberOfRecommendations]
    print(recommended)
    recommendedItems = list(map(lambda x: x[0], recommended))
    if(len(recommendedItems)<numberOfRecommendations):
        recommendedItems=fillWithTopPop(recommendedItems, user)
    return recommendedItems    



In [10]:
print("After function:")
print(getRecommended(100))
getRecommended(151)

After function:
[(32777, 30.895391586032815), (32799, 30.830589722864499), (52, 30.266182654436161), (24649, 29.247348538597937), (16414, 29.193317648687248)]
[32777, 32799, 52, 24649, 16414]
[(16484, 38.43484362595288), (24639, 33.956325093862247), (12354, 33.217055183901067), (12362, 32.899398349217542), (12384, 32.790049158492764)]


[16484, 24639, 12354, 12362, 12384]

In [11]:
f=open("predictionsPip2.csv",'w')
i=0
for user in targets:
    print(user)
    f.write("{}".format(user)+',')
    rec=getRecommended(user)
    print(rec)
    for prod in rec:
        f.write(str(prod) +' ')
    f.write('\n')
    i=i+1
    print(i,"of", len(targets), "written")
f.close()

4
[(32578, 10.645831795733727), (35061, 10.146160793559531), (98, 8.3772621380919716), (30408, 8.0441798186082032), (10129, 7.8069883454017415)]
[32578, 35061, 98, 30408, 10129]
1 of 4196 written
5
[(32799, 35.251438034770054), (52, 34.687030966341716), (32808, 33.197820799238123), (19913, 30.881362592063326), (24595, 30.539405026578095)]
[32799, 52, 32808, 19913, 24595]
2 of 4196 written
8
[(29, 40.306866510280599), (19642, 38.386168271254995), (4, 37.626079482689086), (39, 35.187179935371212), (1304, 35.005160048324193)]
[29, 19642, 4, 39, 1304]
3 of 4196 written
9
[(16414, 35.596039645415516), (10246, 32.735823372939429), (30287, 32.563257259390994), (14381, 31.907457523221108), (12298, 31.117427099367863)]
[16414, 10246, 30287, 14381, 12298]
4 of 4196 written
13
[(16419, 32.864649606264102), (28688, 31.059285661386408), (32775, 30.729155586917937), (24647, 30.076857211398533), (32877, 28.107216224928234)]
[16419, 28688, 32775, 24647, 32877]
5 of 4196 written
18
[(33215, 62.10222749

43 of 4196 written
144
[(14079, 7.4648945931547415), (36496, 7.2786841081793821), (1756, 7.2367887119995338), (31272, 7.2367887119995338), (34813, 7.0598560447574288)]
[14079, 36496, 1756, 31272, 34813]
44 of 4196 written
145
[(18439, 5.8513579763510375), (31345, 5.8513579763510375), (20496, 5.6710027118602433), (13040, 5.4173048254702305), (25777, 5.4173048254702305)]
[18439, 31345, 20496, 13040, 25777]
45 of 4196 written
146
[(21412, 23.69013692519853), (8394, 19.257951046927268), (12873, 12.020735593271191), (3398, 10.855795130376684), (17648, 9.1120084289394558)]
[21412, 8394, 12873, 3398, 17648]
46 of 4196 written
151
[(16484, 38.43484362595288), (24639, 33.956325093862247), (12354, 33.217055183901067), (12362, 32.899398349217542), (12384, 32.790049158492764)]
[16484, 24639, 12354, 12362, 12384]
47 of 4196 written
152
[(72, 375.89506209790056), (5280, 375.89506209790056), (5457, 375.89506209790056), (9489, 375.89506209790056), (10465, 375.89506209790056)]
[72, 5280, 5457, 9489, 10

85 of 4196 written
281
[(15973, 43.067509263512825), (20941, 43.067509263512825), (8216, 41.671652735452277), (17756, 40.933537324485805), (27753, 37.851364083487631)]
[15973, 20941, 8216, 17756, 27753]
86 of 4196 written
288
[(21874, 8.4499928721136151), (36587, 8.4030205210204958), (4034, 8.2619487480718092), (18106, 7.5740168089260846), (20101, 7.2002211217607357)]
[21874, 36587, 4034, 18106, 20101]
87 of 4196 written
294
[(8205, 35.234966183565795), (16367, 34.936094632851194), (8216, 34.110904451110727), (16419, 33.988620124283116), (8206, 33.351561048712604)]
[8205, 16367, 8216, 16419, 8206]
88 of 4196 written
296
[(18741, 17.466330073765672), (35061, 8.458695114002925), (17644, 7.3563798741746478), (25694, 6.4642543540571031), (8809, 6.2567686537326468)]
[18741, 35061, 17644, 25694, 8809]
89 of 4196 written
299
[(8214, 38.118979100056904), (36872, 35.143723437615876), (24613, 34.617435162507164), (8205, 33.917969498946839), (12, 33.555957159539645)]
[8214, 36872, 24613, 8205, 12

[4, 8234, 16402, 48, 24586]
130 of 4196 written
433
[(11758, 41.847404233272229), (28701, 38.630397790516724), (24608, 38.355142382717723), (9, 36.626190535969485), (24595, 34.534661051084399)]
[11758, 28701, 24608, 9, 24595]
131 of 4196 written
441
[(12288, 38.651147862122833), (24595, 36.603979673746501), (8205, 36.441892867045645), (16396, 35.71382450873628), (32777, 34.770400353566714)]
[12288, 24595, 8205, 16396, 32777]
132 of 4196 written
443
[(24608, 36.398951505732704), (54, 36.375665167935438), (57, 35.594679160505144), (15973, 34.74067066244222), (24626, 33.624425177135478)]
[24608, 54, 57, 15973, 24626]
133 of 4196 written
445
[(24626, 39.936692085158839), (24639, 39.270945429807767), (8216, 33.041509820657581), (8205, 33.036680470174197), (32806, 30.278882926987354)]
[24626, 24639, 8216, 8205, 32806]
134 of 4196 written
449
[(28688, 37.654838506072963), (14372, 36.372078963145782), (26639, 27.715622974917899), (28691, 27.408763852424372), (16394, 26.505024633604322)]
[28688

[28688, 34867, 20484, 10927, 32808]
173 of 4196 written
599
[(29, 34.227809754582132), (32777, 32.667737133740296), (32806, 32.052003214837534), (53, 30.521390414794112), (32770, 28.973166471522511)]
[29, 32777, 32806, 53, 32770]
174 of 4196 written
600
[(16632, 18.596140545584742), (1489, 14.959859960731777), (27071, 13.207915095901301), (3067, 11.430566301599139), (22061, 11.374770517333168)]
[16632, 1489, 27071, 3067, 22061]
175 of 4196 written
603
[(32770, 33.759856536320967), (24595, 32.707952872654033), (32808, 32.548919808339633), (24608, 30.722529333888851), (20484, 30.022113859806694)]
[32770, 24595, 32808, 24608, 20484]
176 of 4196 written
605
[(8238, 34.83891465277614), (4105, 31.275174760511181), (28688, 30.360412208887425), (8205, 29.508689255782798), (36872, 28.99844563677572)]
[8238, 4105, 28688, 8205, 36872]
177 of 4196 written
606
[(24608, 33.655802765029485), (12295, 32.812452677731571), (28707, 32.22843972891539), (28701, 31.789353605912147), (32777, 27.4175515631811

...

3994 of 4196 written
14680
[(8205, 32.323537243054517), (28688, 32.269670277997292), (24626, 31.905735634237981), (12384, 31.783574290930279), (4783, 30.296474792762627)]
[8205, 28688, 24626, 12384, 4783]
3995 of 4196 written
14681
[(72, 420.17870208791362), (5280, 420.17870208791362), (5457, 420.17870208791362), (9489, 420.17870208791362), (10465, 420.17870208791362)]
[72, 5280, 5457, 9489, 10465]
3996 of 4196 written
14682
[(8759, 27.58455154906563), (32674, 27.58455154906563), (27131, 11.823923897035682), (33398, 11.212652284114032), (718, 10.12295026207876)]
[8759, 32674, 27131, 33398, 718]
3997 of 4196 written
14687
[(8260, 35.016325964710418), (32777, 33.156069384520613), (32806, 31.911613551839832), (22579, 29.449204032300592), (24626, 28.732450535126201)]
[8260, 32777, 32806, 22579, 24626]
3998 of 4196 written
14692
[(26658, 7.9043290155251951), (14015, 7.1131516464394586), (19844, 5.4811571774309575), (9833, 5.2484465048635309), (6254, 5.1794524176430201)]
[26658, 14015, 19844

4037 of 4196 written
14803
[(128, 32.867475392767403), (24612, 30.970029017179073), (9564, 28.375605074923005), (2748, 27.7249459451644), (11977, 27.695119553992022)]
[128, 24612, 9564, 2748, 11977]
4038 of 4196 written
14809
[(51, 9.8016481277625616), (7357, 9.0441761196915227), (22466, 7.6252658275739229), (21769, 7.0578094755083143), (21687, 7.0314892356180589)]
[51, 7357, 22466, 21769, 21687]
4039 of 4196 written
14813
[(15687, 40.043054181208085), (24626, 37.808146792392336), (54, 36.555967318157975), (32780, 32.071281937626139), (12290, 31.230298310998098)]
[15687, 24626, 54, 32780, 12290]
4040 of 4196 written
14815
[(36533, 14.460153236182137), (30923, 13.572206717772088), (26191, 10.974801608754333), (10067, 8.0908626376019903), (25988, 8.0908626376019903)]
[36533, 30923, 26191, 10067, 25988]
4041 of 4196 written
14821
[(18314, 9.6255695093224283), (3716, 9.235113918452873), (5075, 8.6093710979976983), (28182, 8.6093710979976983), (28573, 8.6093710979976983)]
[18314, 3716, 5075

14952
[(16443, 33.422248164174384), (25642, 31.684901347131355), (32780, 30.951670356474445), (4102, 29.148999940595576), (12352, 28.294298320679452)]
[16443, 25642, 32780, 4102, 12352]
4083 of 4196 written
14953
[(4475, 21.043343851410306), (36889, 8.315780153246763), (37132, 8.2899214456289929), (31076, 8.0803179425713783), (11758, 8.018243092636645)]
[4475, 36889, 37132, 31076, 11758]
4084 of 4196 written
14955
[(17605, 7.7226982770867831), (2442, 7.5016614447431751), (5644, 7.5016614447431751), (21081, 7.5016614447431751), (19452, 7.287848138049295)]
[17605, 2442, 5644, 21081, 19452]
4085 of 4196 written
14960
[(32770, 31.494552170194854), (8205, 29.997305395855939), (32868, 28.796923481473385), (61, 26.977892079251454), (4, 24.835137651007599)]
[32770, 8205, 32868, 61, 4]
4086 of 4196 written
14970
[(16425, 36.035483147930059), (32827, 35.764800844466521), (8205, 34.924091660418171), (24639, 34.167356748440881), (12028, 31.602485041823829)]
[16425, 32827, 8205, 24639, 12028]
4087 

4123 of 4196 written
15096
[(20556, 42.333677485045285), (16402, 36.78429181144768), (8233, 36.194604866219017), (42, 33.794926773489067), (49, 32.907844930707938)]
[20556, 16402, 8233, 42, 49]
4124 of 4196 written
15100
[(15823, 32.818145650634733), (29355, 32.487325438991391), (19913, 30.313543866913356), (11919, 27.98932575004261), (12303, 27.98932575004261)]
[15823, 29355, 19913, 11919, 12303]
4125 of 4196 written
15103
[(24626, 34.632779309397911), (32777, 33.160802866402676), (29, 30.610362963726466), (8216, 29.908169628978609), (4, 29.596239995439291)]
[24626, 32777, 29, 8216, 4]
4126 of 4196 written
15108
[(8469, 12.621965467198208), (13986, 7.436346021399074), (28423, 6.8261843144749932), (32846, 6.8261843144749932), (22677, 6.4564732456598151)]
[8469, 13986, 28423, 32846, 22677]
4127 of 4196 written
15109
[(14996, 10.746341570898288), (2038, 5.5939255279950064), (12969, 5.5821737315943292), (6127, 4.6428240094385904), (8106, 4.6428240094385904)]
[14996, 2038, 12969, 6127, 810

15248
[(52, 35.726979184718978), (32799, 35.607633630051161), (39, 35.556702168254873), (32790, 34.880139379732384), (9, 32.813966939036973)]
[52, 32799, 39, 32790, 9]
4168 of 4196 written
15250
[(16451, 34.937494813237137), (8216, 34.831374861143779), (54, 33.538923575131683), (2041, 31.444067258610897), (15973, 30.459630319323434)]
[16451, 8216, 54, 2041, 15973]
4169 of 4196 written
15251
[(8216, 34.563755992750806), (12295, 31.961357498744135), (28705, 30.452443542799351), (15973, 30.215355495704081), (20941, 30.215355495704081)]
[8216, 12295, 28705, 15973, 20941]
4170 of 4196 written
15269
[(13003, 7.0287595357330837), (9520, 6.5544127594772403), (1480, 6.1328058582002392), (21661, 6.1328058582002392), (23012, 5.8063250455071884)]
[13003, 9520, 1480, 21661, 23012]
4171 of 4196 written
15270
[(24609, 32.948296418228814), (20518, 32.414703553669575), (12320, 32.398243460777941), (4105, 30.379097334699249), (12318, 30.233240711313098)]
[24609, 20518, 12320, 4105, 12318]
4172 of 4196 w

In [18]:
print("Result of getRecommended:\n",getRecommended(151))
print("Top 5 from the matrix: " ,prediction.getrow(151).toarray()[0])

[(8013, 27.457348334313153), (30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616)]
Result of getRecommended:
 [8013, 30796, 12535, 4223, 11554]
Top 5 from the matrix:  [ 0.          3.52206684  6.39351585 ...,  0.58138567  4.65589395  0.        ]


In [19]:
print("Result of getRecommended:\n",getRecommended(100))
print("Top 5 from the matrix: " ,prediction.getrow(100).toarray()[0])

[(12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616), (20532, 13.00817618709717), (19267, 12.695308889326265)]
Result of getRecommended:
 [12535, 4223, 11554, 20532, 19267]
Top 5 from the matrix:  [ 0.          2.36576972  3.58037592 ...,  0.          1.72884326  0.        ]
